In [2]:
data = open('data/pride_and_prejudice.txt', 'r').read()

chars = list(set(data)) 
data_size, vocab_size = len(data), len(chars)
print ('data has ', data_size, 'chars,' ,vocab_size, 'unique')

data has  684782 chars, 76 unique


In [3]:
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i, ch in enumerate(chars)}
print (char_to_ix)
print (ix_to_char)

{'7': 0, '”': 1, 'o': 2, 'v': 3, '?': 4, '“': 5, ';': 6, ')': 7, ' ': 8, 'm': 9, 'Y': 10, '.': 11, 'R': 12, 'x': 13, 'a': 14, 't': 15, ',': 16, '3': 17, 'N': 18, 'w': 19, '6': 20, 'I': 21, 'V': 22, 'u': 23, 'S': 24, 'P': 25, 'h': 26, 'd': 27, 'C': 28, '1': 29, 'j': 30, '!': 31, 's': 32, 'H': 33, 'F': 34, '9': 35, 'e': 36, 'f': 37, 'L': 38, 'Z': 39, 'J': 40, "'": 41, ':': 42, 'g': 43, 'G': 44, 'O': 45, 'D': 46, 'n': 47, 'r': 48, 'l': 49, 'W': 50, '-': 51, 'q': 52, 'z': 53, 'B': 54, 'b': 55, '2': 56, '5': 57, '8': 58, '0': 59, 'U': 60, 'c': 61, 'M': 62, '4': 63, 'y': 64, '*': 65, 'T': 66, '\n': 67, 'E': 68, '_': 69, '(': 70, 'i': 71, 'A': 72, 'k': 73, 'p': 74, 'K': 75}
{0: '7', 1: '”', 2: 'o', 3: 'v', 4: '?', 5: '“', 6: ';', 7: ')', 8: ' ', 9: 'm', 10: 'Y', 11: '.', 12: 'R', 13: 'x', 14: 'a', 15: 't', 16: ',', 17: '3', 18: 'N', 19: 'w', 20: '6', 21: 'I', 22: 'V', 23: 'u', 24: 'S', 25: 'P', 26: 'h', 27: 'd', 28: 'C', 29: '1', 30: 'j', 31: '!', 32: 's', 33: 'H', 34: 'F', 35: '9', 36: 'e', 

In [4]:
import numpy as np

vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['a']] = 1
print (vector_for_char_a.ravel())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [25]:
hidden_size = 10
seq_length = 25
learning_rate = 1e-1

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 #input to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01 #input to hidden
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [26]:
def lossFun(inputs, targets, hprev):
  """                                                                                                                                                                                         
  inputs,targets are both list of integers.                                                                                                                                                   
  hprev is Hx1 array of initial hidden state                                                                                                                                                  
  returns the loss, gradients on model parameters, and last hidden state                                                                                                                      
  """
  #store our inputs, hidden states, outputs, and probability values
  xs, hs, ys, ps, = {}, {}, {}, {} #Empty dicts
    # Each of these are going to be SEQ_LENGTH(Here 25) long dicts i.e. 1 vector per time(seq) step
    # xs will store 1 hot encoded input characters for each of 25 time steps (26, 25 times)
    # hs will store hidden state outputs for 25 time steps (100, 25 times)) plus a -1 indexed initial state
    # to calculate the hidden state at t = 0
    # ys will store targets i.e. expected outputs for 25 times (26, 25 times), unnormalized probabs
    # ps will take the ys and convert them to normalized probab for chars
    # We could have used lists BUT we need an entry with -1 to calc the 0th hidden layer
    # -1 as  a list index would wrap around to the final element
  xs, hs, ys, ps = {}, {}, {}, {}
  #init with previous hidden state
    # Using "=" would create a reference, this creates a whole separate copy
    # We don't want hs[-1] to automatically change if hprev is changed
  hs[-1] = np.copy(hprev)
  #init loss as 0
  loss = 0
  # forward pass                                                                                                                                                                              
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation (we place a 0 vector as the t-th input)                                                                                                                     
    xs[t][inputs[t]] = 1 # Inside that t-th input we use the integer in "inputs" list to  set the correct
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state                                                                                                            
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars                                                                                                           
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars                                                                                                              
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)                                                                                                                       
  # backward pass: compute gradients going backwards    
  #initalize vectors for gradient values for each set of weights 
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    #output probabilities
    dy = np.copy(ps[t])
    #derive our first gradient
    dy[targets[t]] -= 1 # backprop into y  
    #compute output gradient -  output times hidden states transpose
    #When we apply the transpose weight matrix,  
    #we can think intuitively of this as moving the error backward
    #through the network, giving us some sort of measure of the error 
    #at the output of the lth layer. 
    #output gradient
    dWhy += np.dot(dy, hs[t].T)
    #derivative of output bias
    dby += dy
    #backpropagate!
    dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
    dbh += dhraw #derivative of hidden bias
    dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
    dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
    dhnext = np.dot(Whh.T, dhraw) 
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [27]:
#prediction, one full forward pass
def sample(h, seed_ix, n):
  """                                                                                                                                                                                         
  sample a sequence of integers from the model                                                                                                                                                
  h is memory state, seed_ix is seed letter for first time step   
  n is how many characters to predict
  """
  #create vector
  x = np.zeros((vocab_size, 1))
  #customize it for our seed char
  x[seed_ix] = 1
  #list to store generated chars
  ixes = []
  #for as many characters as we want to generate
  for t in range(n):
    #a hidden state at a given time step is a function 
    #of the input at the same time step modified by a weight matrix 
    #added to the hidden state of the previous time step 
    #multiplied by its own hidden state to hidden state matrix.
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    #compute output (unnormalised)
    y = np.dot(Why, h) + by
    ## probabilities for next chars
    p = np.exp(y) / np.sum(np.exp(y))
    #pick one with the highest probability 
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    #create a vector
    x = np.zeros((vocab_size, 1))
    #customize it for the predicted char
    x[ix] = 1
    #add it to the list
    ixes.append(ix)

  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print ('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(hprev,char_to_ix['a'],200)

----
 8tRVn;Os,'rTC'5,m”_3quVF?'ARM6j..3_g1'6(qUG6)um-xHtucV);qg,e*vtK6P;i).s;y'F-OmnP“!l5Zs uPrne*OH)AhLMz”(nGZLH?3_:ukz0;?,.JgpV29pA'q!lZ.wPqs612“5Jb;_q* AHk*Soh-oor'WBAco8a
dm8jJ0kA”,OrG0NL :B“GO1TK6nxSy 
----


In [28]:
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print ("inputs: ",inputs)
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print ("targets: ",targets)

inputs:  [67, 25, 12, 21, 46, 68, 8, 72, 18, 46, 8, 25, 12, 68, 40, 60, 46, 21, 28, 68, 67, 67, 54, 64, 8]
targets:  [25, 12, 21, 46, 68, 8, 72, 18, 46, 8, 25, 12, 68, 40, 60, 46, 21, 28, 68, 67, 67, 54, 64, 8, 40]


In [29]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*150:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # check "How to feed the loss function to see how this part works
  if p+seq_length+1 >= len(data) or n == 0:
    hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
    p = 0 # go from start of data                                                                                                                                                             
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # forward seq_length characters through the net and fetch gradient                                                                                                                          
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001

  # sample from the model now and then                                                                                                                                                        
  if n % 1000 == 0:
    print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
    sample(hprev, inputs[0], 200)

  # perform parameter update with Adagrad                                                                                                                                                     
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

  p += seq_length # move data pointer                                                                                                                                                         
  n += 1 # iteration counter

iter 0, loss: 108.268332
----
 uZlRSkO5tKmit*,e“ONNZ!DZ0nC akVgG)AnGNddahEAxYH“EjAGVorBxyYZp4Tq:_3G”mgURPU'!3iKSCb g-E7DjUdJU)EFvZtcvgq!z9-e”0eH!rCd-!)“IvAVc6D“YAOc”lAGN0YGH9 fAI5”,Cf(;lWP63Unf-GhHq.hi97””I2“U OMrwi?:46S0,'V'h!P.wn 
----
iter 1000, loss: 79.651707
----
 P Mren, bwabind tabivt th hho wheiwhend ted thr bo wweso wind hisouy af ton sesayl. Buto,pissinitheest hasa fenteitheyonson hise, fang iticad am. Be ave whous re thed at Mrnintras qhet

fa finptnhe, l 
----
iter 2000, loss: 67.108263
----
 oft.

“I othas wanve maber whe the, Ble hheowevsethe sur ias thit” oxxmare oA ifhlas the, h; so Uthln basme he th her” th and hond ina My ef ifwart,mesy fryudre lo hind, wherte Mud an an. wang he dint 
----
iter 3000, loss: 61.681601
----
 otis wur'snelithery en herl,
bd psoof on soemus pitcly id yuot pis iey”

facd, and in shod_ repin, lurlril Therdolice thhe maraped ua
pang I shintist by arod qfoacash Moowy, inimerd yh yo Ne hingu ues 
----
iter 4000, loss: 59.539161
----
 Thon y da

KeyboardInterrupt: 